In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error,median_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
#!pip install catboost
from catboost import CatBoostRegressor
import os 
# Monter le Drive
from google.colab import drive
drive.mount('/content/drive')
# Changer de répertoire vers SteamAnalyse
project_dir = '/content/drive/MyDrive/SteamAnalyse'
os.chdir(project_dir)

path="train.parquet"
path_test="test.parquet"

In [ ]:
df = pd.read_parquet(path)
dft = pd.read_parquet(path_test)

In [4]:
def clean_text_col(x):
    if isinstance(x, (list, np.ndarray)):
        return " ".join([str(i) for i in x])
    return str(x) if pd.notna(x) else ""

In [ ]:
cols_text = ["categories","genres","tags_utilisateurs","langues_partiel","langues_complet","description_detaillee"]
numeric_cols = ["prix","nombre_succes","N_langues_complet","N_langues_partiel"]

# Vérification que les colonnes existent
features = numeric_cols + cols_text
target = "review"

X_train = df[features].copy()
for c in cols_text: X_train[c] = X_train[c].apply(clean_text_col)
y_train = np.log1p(df[target])

In [ ]:
cols_text = ["categories","genres","tags_utilisateurs","langues_partiel","langues_complet","description_detaillee"]
numeric_cols = ["prix","nombre_succes","N_langues_complet","N_langues_partiel"]

# Vérification que les colonnes existent
features = numeric_cols + cols_text
target = "review"

X_test = dft[features].copy()
for c in cols_text: X_test[c] = X_test[c].apply(clean_text_col)
y_test = df[target]

In [ ]:
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [ ]:
# 4. CatBoost avec gestion du Texte
cb = CatBoostRegressor(
    depth=12,
    text_features=cols_text,
    verbose=100,
    task_type="GPU"
)

print(f"Entraînement sur le texte : {cols_text}...")
cb.fit(X_train, y_train, eval_set=(X_test, y_test))


# 5. Résultats
y_pred_log = cb.predict(X_test)
score_r2 = r2_score(np.log1p(y_test), y_pred_log)

y_pred = np.expm1(y_pred_log)
print(f"\n🏆 SCORE FINAL (R2 Log) : {score_r2:.4f}")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
score_r2_train = r2_score(y_train, cb.predict(X_train))
print(f"\n SCORE TRAIN (R2 Log) : {score_r2_train:.4f}")
print("Erreur Médiane :", median_absolute_error(y_test, y_pred))

imp = cb.get_feature_importance(prettified=True)
print("\nTop Facteurs de Succès :")
print(imp.head(10))


plt.figure(figsize=(10, 6))
sns.scatterplot(x=np.log1p(y_test), y=y_pred_log, alpha=0.3, color='blue')
plt.plot([0, np.log1p(y_test).max()], [0, np.log1p(y_test).max()], 'r--', lw=2)
plt.title(f"Prédiction du Succès (R2: {score_r2:.3f}) - categories + genres + tags + langue + description_detaillee inclus", fontsize=14)
plt.xlabel("Vrai log(Avis)")
plt.ylabel("Prédiction log(Avis)")
plt.show()

results = pd.DataFrame({
    'Titre': df.loc[X_test.index, 'nom'] if 'nom' in df.columns else X_test.index,
    'Vrai_Avis': y_test,
    'Prediction': y_pred
})

print("Exemples de prédictions :")
print(results.sample(10))
feature_importance = cb.get_feature_importance(prettified=True)
print(feature_importance)